## Fuel Efficiency Regression
Predict a car's fuel efficiency / MPG

In [3]:
import pandas as pd
from importlib import reload
from sandbox import model, layers, activations, costs, utils

In [13]:
# Fetch fuel efficiency data
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

# Remove missing values, drop column I don't want to deal with
dataset = raw_dataset.copy()
dataset = dataset.dropna().drop('Origin', axis=1)

# Seperate labels from features
features = dataset.copy()
labels = dataset.pop('MPG')

# Normalize feature set
features = (features - features.mean()) / features.std()

# Normalize label set
labels_mean = labels.mean()
labels_std = labels.std()
labels = (labels - labels_mean) / labels_std

# Convert dataframes to CuPy arrays
input = features.to_numpy()
labels = labels.to_numpy().reshape(-1, 1)

# Split data into training and test sets
train_x = input[50:, :]
train_y = labels[50:, :]
test_x = input[:50, :]
test_y = labels[:50, :]

In [16]:
reload(model)

# Create model
mpg = model.Model()
mpg.add(layers.Dense(units=16, activation=activations.ReLU()))
mpg.add(layers.Dense(units=4, activation=activations.ReLU()))
mpg.add(layers.Dense(units=1, activation=activations.Linear()))
mpg.configure(learning_rate=0.3, epochs=2000, cost_type=costs.MSE())

# Train model
mpg.train(train_x, train_y)

In [17]:
pred = mpg.predict(test_x)
loss = costs.MSE().forward(pred.T, test_y.T)
print('Test MSE loss: ', loss)

# Note - because the labels were normalized, model outputs must be adjusted before being used, as so:
pred_actual = pred * labels_std + labels_mean

Test MSE loss:  0.08472945714319007
